# Target Value

Apply the pre-trained object detection model to all MSCOCO data and acquire the follow values:

 - Classification false positive, false negative, true positive
 - Mean Average Precision (mAP)


In [1]:
!pip install torchvision==0.13
!pip install pycocotools

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Using cached torchvision-0.13.0-cp38-cp38-manylinux1_x86_64.whl (19.1 MB)
  Using cached torch-1.12.0-cp38-cp38-manylinux1_x86_64.whl (776.3 MB)
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [2]:
from torchvision.models.detection.mask_rcnn import maskrcnn_resnet50_fpn_v2
import torch
print(torch.__version__)

1.12.0+cu102


In [3]:
model = maskrcnn_resnet50_fpn_v2(weights='COCO_V1')
model.eval()

MaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
         

In [4]:
from pycocotools.coco import COCO
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [5]:
dataDir='data'
dataType='train2017'
annFile='{}/annotations/instances_{}.json'.format(dataDir,dataType)
coco=COCO(annFile)

loading annotations into memory...
Done (t=98.15s)
creating index...
index created!


In [6]:
# display COCO categories and supercategories
cats = coco.loadCats(coco.getCatIds())
nms=[cat['name'] for cat in cats]
print('COCO categories: \n{}\n'.format(', '.join(nms)))
nms = set([cat['supercategory'] for cat in cats])
print('COCO supercategories: \n{}'.format(', '.join(nms)))

COCO categories: 
person, bicycle, car, motorcycle, airplane, bus, train, truck, boat, traffic light, fire hydrant, stop sign, parking meter, bench, bird, cat, dog, horse, sheep, cow, elephant, bear, zebra, giraffe, backpack, umbrella, handbag, tie, suitcase, frisbee, skis, snowboard, sports ball, kite, baseball bat, baseball glove, skateboard, surfboard, tennis racket, bottle, wine glass, cup, fork, knife, spoon, bowl, banana, apple, sandwich, orange, broccoli, carrot, hot dog, pizza, donut, cake, chair, couch, potted plant, bed, dining table, toilet, tv, laptop, mouse, remote, keyboard, cell phone, microwave, oven, toaster, sink, refrigerator, book, clock, vase, scissors, teddy bear, hair drier, toothbrush

COCO supercategories: 
food, electronic, outdoor, appliance, accessory, vehicle, indoor, furniture, kitchen, person, animal, sports


In [7]:
# get all images containing given categories, select one at random
catIds = coco.getCatIds(catNms=['person']);
imgIds = coco.getImgIds(catIds=catIds);
print(len(imgIds))
imgs = coco.loadImgs(imgIds)

64115


In [8]:
from torchvision import transforms
from torch.autograd import Variable
torch.no_grad()

## Define Config and DataSet

If we load all images into memory directly, the server will crash. (Tried it, it actually does crash.)

We will need to design config and dataset as this reference:

https://github.com/multimodallearning/pytorch-mask-rcnn/blob/master/coco.py

In [ ]:
# data_transforms = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize([0.5,0.5,0.5], [0.5,0.5,0.5])
# ])

In [ ]:
# images = []
# i = 0
# for img in imgs:
#     i += 1
#     if i % 500 == 0:
#         print(i)
#     file_name = img['file_name']
#     image = Image.open(f'{dataDir}/images/{dataType}/{file_name}')
#     try:
#         imgblob = data_transforms(image)
#         images.append(imgblob)
#     except:
#         print(file_name)

In [ ]:
# images = torch.tensor(images)
# images.shape